In [1]:
import re
import pickle
from heapq import nlargest
import time

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import session_info

In [2]:
# Посмотрим версии используемых модулей.
session_info.show()

In [3]:
# Загрузим ранее сохранённые уникальные навазния компаний.
text_unic = pickle.load(open('companies_name_unic.pickle','rb'))

# Загрузим ранее сохранённый tfidfvectorizer.
my_vectorizer = pickle.load(open('vectorizer_2.pickle','rb'))

def sentence_embedding_fl(string, vectorizer):
    tf_vectorizer = vectorizer
    sentence_list = [tf_vectorizer.transform([word]).toarray() for word in string.split()]
        
    result = np.sum(np.array(sentence_list), axis=0)
    if len(result.shape) == 0:
        result = np.zeros((1,9776))
    
    return result

In [4]:
# Загрузим ранее сохранённые эмбеддинги для названий компаний.
name_embeddings = pickle.load(open('name_embeddings.pickle','rb'))

In [5]:
# Топ для вывода и порог для косинусного сходства.
top = 5
treshold_for_cosine_similarity = 0.5

# Функция для поиска топ 5 схожих названий и их вывода.
def find_top5(similarity_list, top, treshold):
    company_names = []
    maximums = nlargest(top, similarity_list)
    maximums_treshold = [max for max in maximums if max >= treshold ]
    if len(maximums_treshold) > 0:
            indexes_set = set()
            for maximum in maximums_treshold:
                indexes = list(np.where(np.array(similarity_list) == maximum)[0])
                for index in indexes:
                    if index not in indexes_set:
                        indexes_set.add(index)
                        company_names.append(text_unic[index])

    if len(company_names) < top:
        result = company_names
    else:
        result = company_names[:top]
        
    
    print("Косинусное сходство ", [round(float(element),3) for element in maximums])
    print()

    if len(result) > 0:
        for name in result:
            print(name)
    else:
        print('Похожих названий, превышающих порог ',treshold, " не обнаружено." ) 

In [6]:
# Время начала обработки поиска. 
start_time = time.time()

# Тестовый текст 1. Сначала возьмём порог 0.5 для косинусного сходства.
my_test_text_1 = 'Darth Vader is a great Sith Lord'

# Получим эмбеддинг тестового текста
text_vector = sentence_embedding_fl(my_test_text_1, my_vectorizer)

# Вычислим косинусное сходство между тестовым текстом и навзаниями компаний.
cosine_similarity_list = [cosine_similarity( text_vector, sentence_vector)[0] for sentence_vector in name_embeddings ]

find_top5(cosine_similarity_list, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.447, 0.447, 0.447, 0.355, 0.316]

Похожих названий, превышающих порог  0.5  не обнаружено.

Время обработки поиска =  10.703


In [7]:
# Видим, что ничего не найдено, так косинусное сходство меньше порога.

In [8]:
# Время начала обработки поиска. 
start_time = time.time()

# Тестовый текст 1. Теперь возьмём порог 0.3.
my_test_text_1 = 'Darth Vader is a great Sith Lord'

# Получим эмбеддинг тестового текста
text_vector = sentence_embedding_fl(my_test_text_1, my_vectorizer)

# Вычислим косинусное сходство между тестовым текстом и навзаниями компаний.
cosine_similarity_list = [cosine_similarity( text_vector, sentence_vector)[0] for sentence_vector in name_embeddings ]

find_top5(cosine_similarity_list, top, 0.3)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.447, 0.447, 0.447, 0.355, 0.316]

Lord Industrial Ltda
Lord India Pvt., Ltd.
Jm Lord International Llc
Greatech Philippines Inc.
Great Sports Infra Pvt., Ltd.

Время обработки поиска =  10.561


In [9]:
# Видим, что к непохожему на название компании тексту нашлись названия, где фигурируют похожие слова "Lord" и "Great".

In [10]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь попробуем текст, который был в датасете.
my_test_text_2 = 'Powermax Rubber Factory'

# Получим эмбеддинг второго тестового текста
text_vector2 = sentence_embedding_fl(my_test_text_2, my_vectorizer)

# Вычислим косинусное сходство между тестовым текстом и навзаниями компаний.
cosine_similarity_list2 = [cosine_similarity( text_vector2, sentence_vector)[0] for sentence_vector in name_embeddings ]

find_top5(cosine_similarity_list2, top, 0.3)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.816, 0.577, 0.577, 0.577, 0.577]

Powermax Rubber Factory
B G N Rubber Factory
Western Rubbers India Pvt., Ltd.
K.S. Rubbers
Imp. Rubbers International Llc

Время обработки поиска =  10.67


In [11]:
# Видим что на первом месте исходный текст (не удивительно), а дальше тексты с похожими словами. 

In [12]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь попробуем текст, не был в датасете, но похож, на то, что был там.
# Оригинал: Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
# Изменённый вариант
my_test_text_3 = 'Well Aero System Lab Industrial Craft'

# Получим эмбеддинг второго тестового текста
text_vector3 = sentence_embedding_fl(my_test_text_3, my_vectorizer)

# Вычислим косинусное сходство между тестовым текстом и навзаниями компаний.
cosine_similarity_list3 = [cosine_similarity( text_vector3, sentence_vector)[0] for sentence_vector in name_embeddings ]

find_top5(cosine_similarity_list3, top, 0.3)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.494, 0.45, 0.408, 0.408, 0.408]

Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Aerocosta Global Systems Inc.
Craft Colombia Sas
Mx Systems International P Ltd.
Craft Argentina S.A.

Время обработки поиска =  10.719


In [13]:
# Видим что на первом месте текст, который был изменён а дальше тексты с похожими словами. 

In [14]:
# Время поиска и вывода похожих названий около 11 секунд. Долго потому что каждый раз поиск запускается для 18022 эмбеддингов.